# Autonomous Agent to Create Analysis Report
Create an autonomous agent using the LangChain framework.

---

## Workflow

We will explore the concept of "Plan and Execute" LangChain agents and their ability to generate insightful analysis reports based on retrieved documents from Deep Lake. This is the workflow we’ll follow in this project:
1. **Saving Documents on Deep Lake**: We will begin by saving documents on Deep Lake, which serves as our knowledge repository. Deep Lake provides information that our agents can leverage for analysis and report generation.
2. **Creating a Document Retrieval Tool**: Next, we will develop a tool that enables our agent to retrieve the most relevant documents from Deep Lake based on a given query.
3. **Using the Plan and Execute Agent**: The core of our project involves employing a "Plan and Execute" agent to devise a plan for answering a specific query about creating an overview of a topic. Our specific objective is to generate a comprehensive outline of recent events related to Artificial Intelligence regulations by governments, but the final agent could also work for other similar objectives as well.

#### **Plan and Execute**
**Plan and Execute** agents are a new type of agent executor offering a different approach than the traditional agents supported in LangChain. These agents are heavily inspired by the BabyAGI framework and the recent Plan-and-Solve paper. The primary goal of "Plan and Execute" agents is to enable more complex long-term planning, even at the cost of making more calls to the language model.
- The **planner** in the "Plan-and-Execute" framework typically utilizes a language model's reasoning ability to plan out steps and handle ambiguity and edge cases.
- The **executor**, initially an Action Agent, takes the planner's high-level objectives (steps) and determines the tools or actions required to accomplish each step.

## Setup

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

## Implementation

### 1. Get the data

In [2]:
from langchain.document_loaders import SeleniumURLLoader

urls = [
    "https://www.artificialintelligence-news.com/2023/05/23/meta-open-source-speech-ai-models-support-over-1100-languages/",
    "https://www.artificialintelligence-news.com/2023/05/18/beijing-launches-campaign-against-ai-generated-misinformation/"
    "https://www.artificialintelligence-news.com/2023/05/16/openai-ceo-ai-regulation-is-essential/",
    "https://www.artificialintelligence-news.com/2023/05/15/jay-migliaccio-ibm-watson-on-leveraging-ai-to-improve-productivity/",
    "https://www.artificialintelligence-news.com/2023/05/15/iurii-milovanov-softserve-how-ai-ml-is-helping-boost-innovation-and-personalisation/",
    "https://www.artificialintelligence-news.com/2023/05/11/ai-and-big-data-expo-north-america-begins-in-less-than-one-week/",
    "https://www.artificialintelligence-news.com/2023/05/11/eu-committees-green-light-ai-act/",
    "https://www.artificialintelligence-news.com/2023/05/09/wozniak-warns-ai-will-power-next-gen-scams/",
    "https://www.artificialintelligence-news.com/2023/05/09/infocepts-ceo-shashank-garg-on-the-da-market-shifts-and-impact-of-ai-on-data-analytics/",
    "https://www.artificialintelligence-news.com/2023/05/02/ai-godfather-warns-dangers-and-quits-google/",
    "https://www.artificialintelligence-news.com/2023/04/28/palantir-demos-how-ai-can-used-military/",
    "https://www.artificialintelligence-news.com/2023/04/26/ftc-chairwoman-no-ai-exemption-to-existing-laws/",
    "https://www.artificialintelligence-news.com/2023/04/24/bill-gates-ai-teaching-kids-literacy-within-18-months/",
    "https://www.artificialintelligence-news.com/2023/04/21/google-creates-new-ai-division-to-challenge-openai/",
]

loader = SeleniumURLLoader(
    urls=urls,
    binary_location=os.environ.get("BROWSER_EXEC_PATH"),
)
docs = loader.load()

print(docs[0])

page_content="Meta’s open-source speech AI models support over 1,100 languages\n\nAbout the Author\n\nBy Ryan Daws |\n\n\n                            Categories:\n                                    \n\nApplications,\n\nArtificial Intelligence,\n\nCompanies,\n\nDevelopment,\n\nMeta (Facebook),\n\nVoice Recognition,\n\nRyan is a senior editor at TechForge Media with over a decade of experience covering the latest technology and interviewing leading industry figures. He can often be sighted at tech conferences with a strong coffee in one hand and a laptop in the other. If it's geeky, he’s probably into it. Find him on Twitter (@Gadget_Ry) or Mastodon (@gadgetry@techhub.social)\n\nAdvancements in machine learning and speech recognition technology have made information more accessible to people, particularly those who rely on voice to access information. However, the lack of labelled data for numerous languages poses a significant challenge in developing high-quality machine-learning model

### 2. Split the docs

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
splitted_docs = text_splitter.split_documents(docs)

### 3. Store the splitted docs at vectorstore with embeddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import DeepLake

embeddings = HuggingFaceEmbeddings()

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = os.environ.get("ACTIVELOOP_ORG_ID")
my_activeloop_dataset_name = "analysis_outline"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# we add all the chunks to the Deep lake
db.add_documents(splitted_docs)

### 4. Create the tool

In [6]:
retriever = db.as_retriever()
retriever.search_kwargs["k"] = 3

# We define some variables that will be used inside our custom tool
CUSTOM_TOOL_DOCS_SEPARATOR = "\n---------------\n"  # how to join together the retrieved docs to form a single string

# This is the function that defines our custom tool that retrieves relevant
# docs from Deep Lake
def retrieve_n_docs_tool(query: str) -> str:
    """Searches for relevant documents that may contain the answer to the query."""
    docs = retriever.get_relevant_documents(query)
    texts = [doc.page_content for doc in docs]
    texts_merged = (
        "---------------\n"
        + CUSTOM_TOOL_DOCS_SEPARATOR.join(texts)
        + "\n---------------"
    )
    return texts_merged

In [7]:
from langchain.agents.tools import Tool

# We create the tool that uses the "retrieve_n_docs_tool" function
tools = [
    Tool(
        name="Search Private Docs",
        func=retrieve_n_docs_tool,
        description="useful for when you need to answer questions \
about current events about Artificial Intelligence",
    )
]

### 5. Create the `PlanAndExecute` agent

In [8]:
from langchain.chat_models import AzureChatOpenAI
from langchain.experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)

# let's create the Plan and Execute agent
model = AzureChatOpenAI(deployment_name="gpt4", temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

### 6. Run the agent

In [9]:
response = agent.run(
    "Write an overview of Artificial Intelligence regulations by governments by country"
)



> Entering new  chain...
steps=[Step(value='Research the current state of Artificial Intelligence (AI) regulations in various countries.'), Step(value='Identify the key countries with significant AI regulations or ongoing discussions about AI regulations.'), Step(value='For each identified country, provide a brief overview of their AI regulations, including any relevant laws, policies, or guidelines.'), Step(value='Organize the information by country, presenting the findings in a clear and concise manner.'), Step(value='Given the above steps taken, provide an overview of Artificial Intelligence regulations by governments by country.\n')]

> Entering new  chain...
Thought: I need to search for information about AI regulations in various countries.
Action:
```
{
  "action": "Search Private Docs",
  "action_input": "current state of AI regulations in various countries"
}
```
Observation: ---------------
EU committees green-light the AI Act

About the Author

By Ryan Daws |


           

In [11]:
from IPython.display import Markdown, display

display(Markdown(response))

Here is an overview of Artificial Intelligence regulations by governments, organized by country:

European Union:
- The European Parliament's Internal Market Committee and Civil Liberties Committee have endorsed the AI Act, which is the first-ever set of rules for AI in Europe.
- The AI Act aims to ensure that AI systems are transparent, accountable, and respect fundamental rights.

United Kingdom:
- The UK has a 'pro-innovation' approach to AI regulation, focusing on fostering innovation while ensuring AI systems are safe, ethical, and respect privacy.
- The UK's approach includes guidelines and frameworks such as the Data Ethics Framework and the AI Council's AI Roadmap, which provide recommendations for the responsible development and deployment of AI.